<a href="https://colab.research.google.com/github/melisnv/Python-Data-Project/blob/main/Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

___


# Text Generation with Neural Networks

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

#### The aim of this project is that the network can learn play writing structure and spacing, just from a character level.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%tensorflow_version 2.x
import tensorflow as tf

## Step 1: The Data

Grabbed any free text from here: https://www.gutenberg.org/

Choosed all of shakespeare's works, mainly for two reasons:

1. Its a large corpus of text, its usually recommended to have at least a source of 1 million characters total to get realistic text generation.

2. It has a very distinctive style. Since the text data uses old style english and is formatted in the style of a stage play, it will be very obvious to us if the model is able to reproduce similar results.

The aim of this mini learning project is to be able to see the recurrent neural network actually duplicate that style just on a character by character basis.

In [2]:
path_to_file = "shakespeare.txt"

In [3]:
text = open(path_to_file, 'r').read()

In [4]:
text[:500]

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bu"

In [5]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [6]:
print(text[140500:141500])

reward.
  HELENA. Inspired merit so by breath is barr'd.
    It is not so with Him that all things knows,
    As 'tis with us that square our guess by shows;
    But most it is presumption in us when
    The help of heaven we count the act of men.
    Dear sir, to my endeavours give consent;
    Of heaven, not me, make an experiment.
    I am not an impostor, that proclaim  
    Myself against the level of mine aim;
    But know I think, and think I know most sure,
    My art is not past power nor you past cure.
  KING. Art thou so confident? Within what space
    Hop'st thou my cure?
  HELENA. The greatest Grace lending grace.
    Ere twice the horses of the sun shall bring
    Their fiery torcher his diurnal ring,
    Ere twice in murk and occidental damp
    Moist Hesperus hath quench'd his sleepy lamp,
    Or four and twenty times the pilot's glass
    Hath told the thievish minutes how they pass,
    What is infirm from your sound parts shall fly,
    Health shall live free, and s

### Understanding unique characters

In [7]:
vocab = sorted(set(text))
vocab

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}']

In [8]:
len(vocab)

84

## Step 2: Text Processing

### Text Vectorization

A neural network can't take in the raw string data, it needed to be assigned numbers to each character. Creating two dictionaries that can go from numeric index to character and character to numeric index.

In [9]:
char_to_index = {char:ind for ind,char in enumerate(vocab)}
char_to_index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [10]:
char_to_index['H']

33

In [11]:
index_to_char = np.array(vocab)
index_to_char[33]

'H'

In [12]:
encoded_text = np.array([char_to_index[c] for c in text])
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

In [13]:
encoded_text.shape

(5445609,)

In [14]:
sample = text[:500]
sample

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bu"

In [15]:
encoded_text[:500]

array([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 12,  0,  1,  1, 31, 73, 70, 68,  1, 61, 56, 64,
       73, 60, 74, 75,  1, 58, 73, 60, 56, 75, 76, 73, 60, 74,  1, 78, 60,
        1, 59, 60, 74, 64, 73, 60,  1, 64, 69, 58, 73, 60, 56, 74, 60,  8,
        0,  1,  1, 45, 63, 56, 75,  1, 75, 63, 60, 73, 60, 57, 80,  1, 57,
       60, 56, 76, 75, 80,  5, 74,  1, 73, 70, 74, 60,  1, 68, 64, 62, 63,
       75,  1, 69, 60, 77, 60, 73,  1, 59, 64, 60,  8,  0,  1,  1, 27, 76,
       75,  1, 56, 74,  1, 75, 63, 60,  1, 73, 64, 71, 60, 73,  1, 74, 63,
       70, 76, 67, 59,  1, 57, 80,  1, 75, 64, 68, 60,  1, 59, 60, 58, 60,
       56, 74, 60,  8,  0,  1,  1, 33, 64, 74,  1, 75, 60, 69, 59, 60, 73,
        1, 63, 60, 64, 73,  1, 68, 64, 62, 63, 75,  1, 57, 60, 56, 73,  1,
       63, 64, 74,  1, 68, 60, 68, 70, 73, 80, 21,  0,  1,  1, 27, 76, 75,
        1, 75, 63, 70, 76,  1, 58, 70, 69, 75, 73, 56, 58, 75, 60, 59,  1,
       75, 70,  1, 75, 63

-----
## Step 3: Creating Batches

Overall the aim to achieve is to have the model predict the next highest probability character given a historical sequence of characters. Its up to the user to choose how long that historic sequence. Too short a sequence and the result don't have enough information (e.g. given the letter "a" , what is the next character) , too long a sequence and the training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. While there is no correct sequence length choice, it should be considered that the text itself, how long normal phrases are in it, and a reasonable idea of what characters/words are relevant to each other.

In [16]:
line = "From fairest creatures we desire increase"

In [17]:
len(line)

41

In [18]:
lines = '''
From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
'''

In [19]:
len(lines)

133

### Training Sequences

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam"
Sequence Out: "ello my name"


Can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

In [20]:
sequence_length = 120

In [21]:
total_num_seq = len(text) // (sequence_length)
total_num_seq

45380

In [22]:
# Create training sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for item in char_dataset.take(500):
    print(item.numpy())

0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
12
0
1
1
31
73
70
68
1
61
56
64
73
60
74
75
1
58
73
60
56
75
76
73
60
74
1
78
60
1
59
60
74
64
73
60
1
64
69
58
73
60
56
74
60
8
0
1
1
45
63
56
75
1
75
63
60
73
60
57
80
1
57
60
56
76
75
80
5
74
1
73
70
74
60
1
68
64
62
63
75
1
69
60
77
60
73
1
59
64
60
8
0
1
1
27
76
75
1
56
74
1
75
63
60
1
73
64
71
60
73
1
74
63
70
76
67
59
1
57
80
1
75
64
68
60
1
59
60
58
60
56
74
60
8
0
1
1
33
64
74
1
75
60
69
59
60
73
1
63
60
64
73
1
68
64
62
63
75
1
57
60
56
73
1
63
64
74
1
68
60
68
70
73
80
21
0
1
1
27
76
75
1
75
63
70
76
1
58
70
69
75
73
56
58
75
60
59
1
75
70
1
75
63
64
69
60
1
70
78
69
1
57
73
64
62
63
75
1
60
80
60
74
8
0
1
1
31
60
60
59
5
74
75
1
75
63
80
1
67
64
62
63
75
5
74
1
61
67
56
68
60
1
78
64
75
63
1
74
60
67
61
9
74
76
57
74
75
56
69
75
64
56
67
1
61
76
60
67
8
0
1
1
38
56
66
64
69
62
1
56
1
61
56
68
64
69
60
1
78
63
60
73
60
1
56
57
76
69
59
56
69
58
60
1
67
64
60
74
8
0
1
1
45
63
80
1
74
60
67
61
1
75
63
80
1
61
70
60
8
1
75
70
1
75
63


In [23]:
#for item in char_dataset.take(500):
    #print(index_to_char[item.numpy()])

The **batch** method converts these individual character calls into sequences that can be feeded in as a batch. Using seq_len+1 because of zero indexing. Here is what drop_remainder means:

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.


In [24]:
sequences = char_dataset.batch(sequence_length+1, drop_remainder=True)

----
Now that there are sequences, will perform the following steps for each one to create the target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

In [25]:
def create_sequence_target(seq):
    
    input_txt = seq[:-1] # Hello my nam
    target_txt = seq[1:] # ello my name
    
    return input_txt, target_txt

In [26]:
dataset =sequences.map(create_sequence_target)

In [27]:
for input_txt,target_txt in dataset.take(1):
    print(input_txt.numpy())
    print("".join(index_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    print("".join(index_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


---
### Generating training batches

Now that there are the actual sequences, creating the batches, shuffle the sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [28]:
batch_size = 128

In [29]:
buffle_size = 10000

dataset = dataset.shuffle(buffle_size).batch(batch_size, drop_remainder= True)

In [30]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

----
## Step 4: Creating the Model

Using an LSTM based model with a few extra features, including an embedding layer to start off with and **two** LSTM layers. This model architecture off the [DeepMoji](https://deepmoji.mit.edu/) and the original source code can be found [here](https://github.com/bfelbo/DeepMoji).

The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. Could be imagined, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

In [31]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding size
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding,Dropout, GRU

### Setting up Loss Function

Sparse categorical crossentropy will be use for the loss.

In [33]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

sparse_categorical_crossentropy : Computes the sparse categorical crossentropy loss.

    Args:
      y_true: Ground truth values.
      y_pred: The predicted values.
      from_logits: Whether `y_pred` is expected to be a logits tensor. By default,
        we assume that `y_pred` encodes a probability distribution.
      axis: (Optional) Defaults to -1. The dimension along which the entropy is
        computed.
    
    Returns:
      Sparse categorical crossentropy loss value.

In [34]:
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits= True)

     Embedding(*args, **kwargs) : Turns positive integers (indexes) into dense vectors of fixed size.

    e.g. `[[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]`

    This layer can only be used as the first layer in a model.

    Args:
      input_dim: Integer. Size of the vocabulary, i.e. maximum integer index + 1.
      output_dim: Integer. Dimension of the dense embedding.
      embeddings_initializer: Initializer for the `embeddings` matrix (see `keras.initializers`).
      embeddings_regularizer: Regularizer function applied to the `embeddings` matrix (see `keras.regularizers`).
      embeddings_constraint: Constraint function applied to the `embeddings` matrix (see `keras.constraints`).

In [35]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    
    model = Sequential()
    
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape = [batch_size, None]))
    
    model.add(GRU(rnn_neurons, 
                  return_sequences = True, 
                  stateful = True, 
                  recurrent_initializer = 'glorot_uniform'))
    
    # Final dense layer to predict
    model.add(Dense(vocab_size))
    
    model.compile('adam', loss = sparse_cat_loss)
    
    return model

In [36]:
model = create_model(vocab_size = vocab_size,
                    embed_dim = embed_dim,
                    rnn_neurons = rnn_neurons,
                    batch_size = batch_size)

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


----
## Step 5: Training the model

Making sure everything is well with the model before the training! Passing in a batch to confirm the model currently predicts random characters without any training.

In [38]:
for input_example_batch, target_example_batch in dataset.take(1):
  
  example_batch_predictions = model(input_example_batch)

In [39]:
example_batch_predictions.shape

TensorShape([128, 120, 84])

In [40]:
 sampled_indices = tf.random.categorical(example_batch_predictions[0],num_samples=1)

In [42]:
sampled_indices

<tf.Tensor: shape=(120, 1), dtype=int64, numpy=
array([[41],
       [46],
       [21],
       [58],
       [36],
       [66],
       [49],
       [63],
       [53],
       [23],
       [52],
       [38],
       [61],
       [17],
       [24],
       [33],
       [21],
       [12],
       [22],
       [59],
       [80],
       [28],
       [51],
       [23],
       [64],
       [10],
       [48],
       [44],
       [36],
       [40],
       [56],
       [37],
       [61],
       [68],
       [55],
       [20],
       [57],
       [ 8],
       [ 7],
       [60],
       [53],
       [81],
       [10],
       [37],
       [ 5],
       [74],
       [51],
       [60],
       [76],
       [73],
       [10],
       [39],
       [60],
       [67],
       [41],
       [60],
       [37],
       [13],
       [58],
       [17],
       [80],
       [32],
       [10],
       [83],
       [32],
       [24],
       [32],
       [23],
       [81],
       [30],
       [22],
       [41],
       [59],
   

    tf.squeeze : Removes dimensions of size 1 from the shape of a tensor.

    Args
    input :	A Tensor. The input to squeeze.
    axis:	An optional list of ints. Defaults to []. 
    If specified, only squeezes the dimensions listed. The dimension index starts at 0. It is an error to squeeze a dimension that is not 1.



In [44]:
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([41, 46, 21, 58, 36, 66, 49, 63, 53, 23, 52, 38, 61, 17, 24, 33, 21,
       12, 22, 59, 80, 28, 51, 23, 64, 10, 48, 44, 36, 40, 56, 37, 61, 68,
       55, 20, 57,  8,  7, 60, 53, 81, 10, 37,  5, 74, 51, 60, 76, 73, 10,
       39, 60, 67, 41, 60, 37, 13, 58, 17, 80, 32, 10, 83, 32, 24, 32, 23,
       81, 30, 22, 41, 59, 41, 25, 64, 44, 26, 32, 60, 51, 49, 14, 37, 18,
       61, 46, 15, 26, 37, 16, 11, 24, 35,  9,  3, 12, 55, 41, 70,  2, 26,
       71, 77, 44, 54, 60, 62, 15, 76,  7, 82, 57, 29, 46, 17, 13, 61, 57,
       69])

In [45]:
index_to_char[sampled_indices]

array(['P', 'U', ':', 'c', 'K', 'k', 'X', 'h', ']', '<', '[', 'M', 'f',
       '6', '>', 'H', ':', '1', ';', 'd', 'y', 'C', 'Z', '<', 'i', '.',
       'W', 'S', 'K', 'O', 'a', 'L', 'f', 'm', '`', '9', 'b', ',', ')',
       'e', ']', 'z', '.', 'L', "'", 's', 'Z', 'e', 'u', 'r', '.', 'N',
       'e', 'l', 'P', 'e', 'L', '2', 'c', '6', 'y', 'G', '.', '}', 'G',
       '>', 'G', '<', 'z', 'E', ';', 'P', 'd', 'P', '?', 'i', 'S', 'A',
       'G', 'e', 'Z', 'X', '3', 'L', '7', 'f', 'U', '4', 'A', 'L', '5',
       '0', '>', 'J', '-', '"', '1', '`', 'P', 'o', '!', 'A', 'p', 'v',
       'S', '_', 'e', 'g', '4', 'u', ')', '|', 'b', 'D', 'U', '6', '2',
       'f', 'b', 'n'], dtype='<U1')

In [47]:
print("Given the input seq: \n")
print("".join(index_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(index_to_char[sampled_indices ]))


Given the input seq: 

ife to Hotspur, and sister to Mortimer.
  Lady Mortimer, daughter to Glendower, and wife to Mortimer.
  Mistress Quickly


Next Char Predictions: 

PU:cKkXh]<[Mf6>H:1;dyCZ<i.WSKOaLfm`9b,)e]z.L'sZeur.NelPeL2c6yG.}G>G<zE;PdP?iSAGeZX3L7fU4AL50>J-"1`Po!ApvS_eg4u)|bDU62fbn


In [48]:
epochs = 30

In [50]:
#model.fit(dataset,epochs= epochs)

In [ ]:
model.summary()

In [ ]:
def generate_text(model, start_seed, gen_size = 100, temp = 1.0):

  '''
  model : Trained model to generate text
  start_seed : Initial seed text in string form
  gen_size : Number of characters to generate

  Basic idea of this function is to take in some seed text, format it so that it is 
  in the correct shape for our network, then loop the sequence as we keep adding our 
  predicted characters. Similar to the work in the RNN time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vectorizing starting seed text
  input_eval = [char_to_index[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval,0)

  # Empty list to hold the resulting generated text
  text_generated = []

  temperature = temp

  # Batch size = 1
  model.reset_states()

  for i in range(num_generate):

    # Generate predictions
    predictions = model(input_eval)

    # Remove the batch shape dimension
    predictions = tf.squeeze(predictions, 0)

    # Use a categorical distribution to select the next character
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples= 1)[-1,0].numpy()

    # Pass the predicted character for the next input
    input_eval = tf.expand_dims([predicted_id],0)

    # Transform back to character letter
    text_generated.append(index_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [ ]:
print(generate_text(model,"flower",gen_size = 1000))